In [132]:
# Import packages
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math

In [133]:
# Import data
data = np.loadtxt('data/MacroSeries.txt', delimiter=',')
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

In [134]:
# All data moments equal 0 by construction
data_moment1 = 0
data_moment2 = 0
data_moment3 = 0
data_moment4 = 0

In [135]:
def model_moments(c,k,w,r,*params):
    alpha, beta, rho, mu = params
    #Use equation 4 to back out the values for z given parameters
    z = np.zeros(len(data))
    for i in range(len(z)):
            z[i] = np.log(r[i]/(alpha*(k[i])**(alpha-1)))
    moments_1 = []
    moments_2 = []
    moments_3 = []
    moments_4 = []
    for i in range(len(z)-1):
        #Equation 6:
        A = z[i+1] - rho*z[i] - (1-rho)*mu
        moments_1.append(A)
        
        #Equation 7:
        B = (z[i+1] - rho*z[i] -(1-rho)*z[i])*z[i]
        moments_2.append(B)
        
        #Equation 8:
        C = (beta*alpha*np.exp(z[i+1])*(k[i+1]**(alpha-1)))*(c[i]/c[i+1]) - 1
        moments_3.append(C)
        
        #Equation 9:
        D = ((beta*alpha*np.exp(z[i+1])*(k[i+1]**(alpha-1)))*(c[i]/c[i+1]) - 1)*w[i]
        moments_4.append(D)
    
    moment_1 = sum(moments_1)/len(moments_1)
    moment_2 = sum(moments_2)/len(moments_2)
    moment_3 = sum(moments_3)/len(moments_3)
    moment_4 = sum(moments_4)/len(moments_4)
    return moment_1, moment_2, moment_3, moment_4

In [136]:
def err_vec(c, k, w, r, *params):
    alpha, beta, rho, mu = params
    moms_data = np.array([ data_moment1, data_moment2, data_moment3, data_moment4])
    moment_1, moment_2, moment_3, moment_4 = model_moments(c,k,w,r, alpha, beta, rho, mu)
    moms_model = np.array([moment_1, moment_2, moment_3, moment_4])
    err_vec = moms_model - moms_data
    return err_vec

In [137]:
def criterion(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, W = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu)
    crit_val = np.dot(np.dot(err.T,W),err)
    return crit_val

In [141]:
#  Parameters initial conditions
params_init = np.array( [0.25 ,0.75, 0, 1] )

#GMM Matrix
W_I = np.eye(4)
# all required components
GMM_args = (c, k, w, r, W_I)
# define params bounds
param_bounds = ( (1e-10, 1-1e-10), (1e-10, 1-1e-10) , \
                 (-1+1e-10, 1-1e-10), (1e-10, None) )

# optimize
results_GMM = opt.minimize(criterion, params_init, args=(c,k,w,r,W_I), \
                           bounds = param_bounds, method = 'L-BFGS-B') 


# print out the results
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results_GMM.x
print(results_GMM)
print(alpha_GMM)
print(beta_GMM)
print(rho_GMM)
print(mu_GMM)
print(results_GMM.fun)

      fun: 0.002797063678308983
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  5.63359078e-02,   2.65498019e+03,  -5.29134028e-06,
         2.60890732e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 205
      nit: 22
   status: 0
  success: True
        x: array([  9.41422436e-01,   9.89999995e-01,   1.81288054e-04,
         9.99817402e-01])
0.94142243623
0.989999995012
0.000181288053965
0.999817401592
0.00279706367831
